In [1]:
import requests
import re
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd

In [2]:
# 의협신문 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.doctorsnews.co.kr/news/articleList.html?page={}&total=117884&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_add_section_code=&sc_add_sub_section_code=&sc_add_serial_code=&sc_area=&sc_level=&sc_m_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=I&view_type=sm'.format(i)
    site = 'http://www.doctorsnews.co.kr/news/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.find('div',{'id':'viewFINE'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages.findAll("a"):
        if 'href' in page.attrs:
            if 'articleView'in page.attrs['href']:
                if (site+page.attrs['href']) not in gisa_links:
                    gisa_links.append(site+page.attrs['href'])
                else:
                    pass
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('td',{'id':'font_date'}).span.get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            origin_title = dom.find('td',{'class':'title'}).get_text()
            re_title = re.sub(title_pattern,' ', origin_title)
            title.append(re_title)
            
            # 부제목 크롤링
            #stitle = dom.find('td',{'class':'sub'}).get_text()
            
            # 내용 크롤링
            origin_content = dom.find('div',{'class':'articleBody'}).get_text()
            re_content = re.sub(content_pattern, ' ', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if int(re_dates[0:4]+re_dates[5:7]) < int(201609):
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_doctorsnews = pd.DataFrame(news_dict)

page1
201711
******************************
page2
201711
******************************
crawling END


In [3]:
# 약업신문 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 0
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'https://www.yakup.com/news/index.html?num_start={}&keyword=&cat=all&cat2=&cat3=&mode=&pmode='.format(16*i)
    site = 'http://www.yakup.com/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.find('ul',{'class':'tp_5'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages.findAll('dt',{'class':'outline'}):
        gisa_links.append(site+page.find("a").attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.findAll('p',{'class':'conTop2'})
            re_dates = re.findall(date_pattern, dates[1].get_text())[0]
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('h4',{'class':'h4Type01'}).get_text())

            # 부제목 크롤링
            #stitle = dom.find('p',{'class':'pType01 tp_7'}).get_text()
            
            # 내용 크롤링                           
            origin_contents = dom.find('div',{'class':'bodyarea'}).get_text()
            re_content = re.sub(content_pattern, ' ', origin_contents)            # lang: 'ko' , 트윗하기 헤결필요
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
    # if int(re_dates[0:4]+re_dates[5:7]) < int(201609):
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_yakup = pd.DataFrame(news_dict)


page0
201711
******************************
page1
201711
******************************
page2
201711
******************************
crawling END


In [14]:
# 의학신문 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.bosa.co.kr/news/articleList.html?page={}&total=292606&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=E&sc_edit=&view_type=sm'.format(i)
    site = 'http://www.bosa.co.kr/news/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.find('div',{'id':'article-control'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages.findAll("a"):
        if 'href' in page.attrs:
            if 'articleView'in page.attrs['href']:
                if (site+page.attrs['href']) not in gisa_links:
                    gisa_links.append(site+page.attrs['href'])
                else:
                    pass
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('li',{'class':'date'}).get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('strong',{'class':'headline-title'}).get_text())
            
            # 부제목 크롤링
            #stitle = dom.find('td',{'class':'sub'}).get_text()
            
            # 내용 크롤링
            origin_content =dom.find('div',{'class':'cont-body'}).get_text()
            re_content = re.sub(content_pattern, ' ', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_bosa = pd.DataFrame(news_dict)

page1
201711
******************************
page2
201711
******************************
crawling END


In [19]:
# 의학뉴스 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s+'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.newsmp.com/news/articleList.html?page=1&total=173428&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=I&view_type='.format(i)
    site = 'http://www.newsmp.com/news/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.findAll('td',{'class':'ArtList_Title'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages:
        gisa_links.append(site+page.find("a").attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('div',{'class':'View_Time'}).get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('div',{'class':'View_Title'}).strong.get_text())
            
            # 부제목 크롤링
            # stitle = dom.find('td',{'class':'sub'}).get_text()
            
            # 내용 크롤링
            origin_content = dom.find('td',{'id':'articleBody'}).get_text()
            re_content = re.sub(content_pattern, ' ', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_newsmp = pd.DataFrame(news_dict)

page1
201711
******************************
page2
201711
******************************
crawling END


In [42]:
# 메디소비자뉴스 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s+'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.medisobizanews.com/news/articleList.html?page={}&total=39519&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=&view_type=sm'.format(i)
    site = 'http://www.medisobizanews.com/news/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.findAll('span',{'style':'font-size:10pt; letter-spacing:-0.3pt;'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages:
        gisa_links.append(site+page.find('a').attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = str(dom.find('td',{'bgcolor':'EFEFEF'}).span.get_text())
            re_dates = dates[0:4] + '-' + dates[6:8] + '-' + dates[10:12]
            date.append(re_dates)
            
            # 제목 크롤링
            origin_title = dom.find('td',{'class':'view_t'}).get_text()
            re_title = re.sub(title_pattern,' ', origin_title)
            title.append(re_title)
            
            # 부제목 크롤링
            #stitle = dom.find('td',{'class':'view_sub_t'}).get_text()
            
            # 내용 크롤링
            origin_content = dom.find('td',{'id':'articleBody'}).get_text()
            re_content = re.sub(content_pattern, ' ', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_medisobizanews = pd.DataFrame(news_dict)

2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-06
2017-11-04
page1
201711
******************************
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-03
2017-11-02
2017-11-02
2017-11-02
2017-11-02
page2
201711
******************************
2017-11-02
2017-11-02
2017-11-02
2017-11-02
2017-11-02
2017-11-02
2017-11-02
2017-11-02
2017-11-02
2017-11-02
2017-11-02
2017-11-02
2017-11-01
2017-11-01
2017-11-01
2017-11-01
2017-11-01
2017-11-01
2017-11-01
2017-11-01
crawling END


In [46]:
# 메디칼옵저버 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    urls = [
        'http://www.monews.co.kr/news/articleList.html?page={}&total=27130&sc_section_code=S1N1&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=I&sc_tags=&view_type=',
        'http://www.monews.co.kr/news/articleList.html?page={}&total=26163&sc_section_code=S1N3&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=I&sc_tags=&view_type=',
        'http://www.monews.co.kr/news/articleList.html?page={}&total=14476&sc_section_code=S1N2&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=I&sc_tags=&view_type='
          ]
    site = 'http://www.monews.co.kr/news/'

    # url 수집
    for url in urls:
        res = requests.get(url)
        dom = BeautifulSoup(res.content, 'html5lib')
        pages = dom.findAll('div',{'class':'list-titles'})
    
        gisa_links = []


        # 페이지 별 기사링크 크롤링
        for page in pages:
            gisa_links.append(site+page.find('a').attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('div',{'class':'View_Time'}).get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('div',{'class','View_Title'}).strong.get_text())
            
            # 부제목 크롤링 -> 없는것도 있음
            #stitle = dom.find('div',{'class','View_Title'}).span.get_text()
            
            # 내용 크롤링
            origin_content = dom.find('td',{'class','view_r'}).get_text()
            re_content = re.sub(content_pattern, ' ', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_monews = pd.DataFrame(news_dict)

page1
201711
******************************
page2
201711
******************************
crawling END


In [48]:
# 약사신문(팜뉴스) 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.pharmnews.co.kr/1news_main.asp?page={}&ng=01&dir=home&part=&s_target=&s_str=&ref_num=&key=&ne='.format(i)
    site = 'http://www.pharmnews.co.kr/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.find('ul',{'class':'main_listview'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages.findAll('a'):
        if 'href' in page.attrs:
            if (site+page.attrs['href'] not in gisa_links):
                gisa_links.append(site+page.attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('div',{'class':'listview_view_header'}).div.get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('div',{'class':'listview_view_header'}).h2.get_text())
            
            # 부제목 크롤링
            #stitle = dom.find('div',{'class':'listview_view_header'}).h3.get_text()
            
            # 내용 크롤링
            origin_content = dom.find('div',{'class':'listview_view_content'}).get_text()
            re_content = re.sub(content_pattern, ' ', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_pharmnews = pd.DataFrame(news_dict)

page1
201711
******************************
page2
201711
******************************
crawling END


In [6]:
# 청년의사 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|(\s\s)+'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.docdocdoc.co.kr/news/articleList.html?page={}&total=81806&sc_section_code=S1N1&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=E&view_type=sm'.format(i)
    site = 'http://www.docdocdoc.co.kr/news/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.find('table',{'id':'article-list'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages.findAll("a"):
        if len(page.attrs['href']) < 100:
            if (site+page.attrs['href']) not in gisa_links:
                gisa_links.append(site+page.attrs['href'])
            else:
                pass
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('li', {'class':'date'}).get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('font',{'class':'headline-title'}).get_text())
            
            # 부제목 크롤링
            #stitle = dom.find('font',{'class':'headline-sub'}).get_text()
            
            # 내용 크롤링
            origin_contents = dom.find('div',{'id':'articleBody'})
            re_contents = origin_contents.findAll('p')
            contents = str()
            for re_content in re_contents:
                contents += re_content.get_text()
            content.append(contents)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>1: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_docdocdoc = pd.DataFrame(news_dict)

page1
201711
******************************
crawling END


In [8]:
# 메디팜스투데이 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s+'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.pharmstoday.com/news/articleList.html?page={}&total=122273&sc_section_code=S1N1&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=R&view_type='.format(i)
    site = 'http://www.pharmstoday.com/news/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.find('div',{'id':'ND_Warp'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages.findAll('a'):
        if len(page.attrs['href']) == 29:
            if (site+page.attrs['href']) not in gisa_links:
                gisa_links.append(site+page.attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = str(dom.find('td',{'bgcolor':'EFEFEF'}).span.get_text())
            re_dates = dates[0:4] + '-' + dates[6:8] + '-' + dates[10:12]
            date.append(re_dates)
            
            # 제목 크롤링
            origin_title = dom.find('td',{'class':'view_t'}).get_text()
            re_title = re.sub(title_pattern, '', origin_title)
            title.append(re_title)
            
            # 부제목 크롤링
            #stitle = dom.find('td',{'class':'sub'}).get_text()
            
            # 내용 크롤링
            origin_content = dom.find('td',{'id':'articleBody'}).get_text()
            re_content = re.sub(content_pattern, '', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_pharmstoday = pd.DataFrame(news_dict)

page1
201711
******************************
page2
201711
******************************
crawling END


In [12]:
# 데일리메디 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s+'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.dailymedi.com/section.php?pg={}&numberpart=&thread=22&pick=&file2=&file=&area=&&user_id=&&user_name=&&start_date=&end_date='.format(i)
    site = 'http://www.dailymedi.com/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.findAll('td',{'class':'smfont_main_title'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages:
        gisa_links.append(site+page.find('a').attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            main = dom.find('div',{'id':'sub_center_contents2'})
            
            # 날짜 크롤링
            dates = main.find('font',{'color':'#666666'}).get_text()
            re_dates = dates[2:6] + '-' + dates[8:10] + '-' + dates[12:14]
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(main.find('td',{'style':'line-height:30px; color:#000000; letter-spacing:-1px; font-size:27px; font-weight:bold; font-family:Dotum,돋움,맑은고딕, MalgunGothic;'}).get_text())
            
            # 부제목 크롤링
            #stitle = main.find('td',{'style':'line-height:40px; color:#000099; font-size:17px; font-weight:bold; font-family:Dotum,돋움,맑은고딕,Malgu Gothic;'}).get_text()
            
            # 내용 크롤링
            origin_content = dom.find('td',{'id':'ct'}).get_text()
            re_content = re.sub(content_pattern, '', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_dailymedi = pd.DataFrame(news_dict)

page1
201711
******************************
page2
201711
******************************
crawling END


In [14]:
# 후생신보 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[/\.-]\d\d[/\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.whosaeng.com/sub_view.html?page={}&local=&section=&section2=&keyword=&level&n_id=&n_name=&t_level='.format(i)
    site = 'http://www.whosaeng.com/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.findAll('div',{'class':'list_area'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages:
        gisa_links.append(site+page.find("a").attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('td',{'class':'data2'}).get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('td',{'class':'read_title'}).get_text())
            
            # 부제목 크롤링
            #stitle = dom.find('td',{'class':'read_subtitle'}).get_text()
            
            # 내용 크롤링
            origin_content = dom.find('div',{'id':'textinput'}).get_text()
            re_content = re.sub(content_pattern, ' ', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_whosaeng = pd.DataFrame(news_dict)

page1
201711
******************************
page2
201711
******************************
crawling END


In [16]:
# 라포르시안 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.rapportian.com/news/articleList.html?page={}&total=32986&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=E&view_type=sm'.format(i)
    site = 'http://www.rapportian.com/news/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.findAll('td',{'class':'list-titles'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages:
        gisa_links.append(site+page.find("a").attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('li',{'class':'date'}).get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('font',{'class':'headline-title'}).get_text())
            
            # 부제목 크롤링 => 없음
            #stitle = dom.find('td',{'class':'read_subtitle'}).get_text()
            
            # 내용 크롤링
            origin_content = dom.find('div',{'id':'articleBody'}).get_text()
            re_content = re.sub(content_pattern, '', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_rapportian = pd.DataFrame(news_dict)

page1
201711
******************************
page2
201711
******************************
crawling END


In [20]:
# 약사공론 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.kpanews.co.kr/article/list.asp?page={}&table=article&category=&search=&keyword='.format(i)
    site = 'http://www.kpanews.co.kr/article/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.findAll('div',{'class':'list_kn'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages:
        gisa_links.append(site+page.find("a").attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('ul',{'class':'kn2'}).li.get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('h2',{'class':'h2 colt_blue'}).get_text())
            
            # 부제목 크롤링
            #stitle = dom.find('h3',{'class':'h3'}).get_text()
            
            # 내용 크롤링
            origin_content = dom.find('div',{'class':'kn2_content'}).get_text()
            re_content = re.sub(content_pattern, '', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_kpanews = pd.DataFrame(news_dict)

page1
201711
******************************
page2
201711
******************************
crawling END


In [22]:
# 헬스포커스뉴스 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.healthfocus.co.kr/news/articleList.html?page=1&total=61420&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=E&view_type=sm'.format(i)
    site = 'http://www.healthfocus.co.kr/news/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.findAll('td',{'class':'list-titles'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages:
        gisa_links.append(site+page.find("a").attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('div',{'class':'info'}).get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('span',{'class':'headline-title'}).get_text())
            
            # 부제목 크롤링 => 없음
            #stitle = dom.find('h3',{'class':'h3'}).get_text()
            
            # 내용 크롤링
            origin_content = dom.find('div',{'id':'articleBody'}).get_text()
            re_content = re.sub(content_pattern, '', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_healthfocus = pd.DataFrame(news_dict)

page1
201711
******************************
page2
201711
******************************
crawling END


In [28]:
# 메디코파마뉴스 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.emedico.co.kr/news/articleList.html?page={}&total=41362&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=&view_type='.format(i)
    site = 'http://www.emedico.co.kr/news/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.findAll('span',{'style':'font-size:10pt; letter-spacing:-0.3pt;'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages:
        gisa_links.append(site+page.find("a").attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('td',{'bgcolor':'EFEFEF'}).get_text()
            re_dates = dates[0:4] + '-' + dates[6:8] + '-' + dates[10:12]
            date.append(re_dates)
            
            # 제목 크롤링
            origin_title = dom.find('td',{'class':'view_t'}).get_text()
            re_title = re.sub(title_pattern,'', origin_title)
            title.append(re_title)
            
            # 부제목 크롤링
            #stitle = dom.find('td',{'class':'view_sub_t'}).get_text()
            
            # 내용 크롤링
            origin_content = dom.find('td',{'id':'articleBody'}).get_text()
            re_content = re.sub(content_pattern, '', origin_content)
            content.append(re_content)
            print(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_emedico = pd.DataFrame(news_dict)

유한양행과 대한 결핵 및 호흡기학회가 제정한 제39회 유한결핵 및 호흡기학술상에 고신의대정만홍 교수와 원광의대 정은택 교수가 수상했다.수상자들은 10일 서울 잠실 롯데호텔에서 열린 제124차 대한결핵 및 호흡기학회 추계학술대회 및 정기총회 석상에서 상패와 상금 및메달을 받았다.수상자들은 탁월한 연구성과로 호흡기 분야 의학발전에 기여한 공로를 인정받았다.한편 지난 1979년 유한양행과 대한결핵 및 호흡기학회가 제정한 유한결핵및호흡기학술상은 매년 이 분야 의학 발전에 공적이 두드러진 의학자에게 수여하는 시상제도로 39년간 호흡기 학술 발전에 큰 기여를 해왔다는 평가 속에 국내 호흡기 분야 최고 권위 학술상으로 자리잡았다.
복지부는 10일 약제 급여 목록 및 급여 상한금액표 고시를 개정하고 오는 15일부터 비소세포폐암 표적치료제 '올리타정'(한미약품)이 건강보험을 적용받을 수 있게 됐다고 밝혔다.'올리타정'은 지난 1일 제18차 건강보험정책심의위원회에서 3상 임상시험을 전제로 조건부 허가된 약제임을 감안해 보험공단과 제약사간 협의를 통해 임상시험기한 불확실성을 해소한 후 차기 건정심에서 서면의결하기로 결정됐다.또한 보험공단-제약사간 추가협의를 통해 협상내용을 보완한 후 제19차 건정심에서 의결됨에 따라 관련 고시 개정이 가능하게 됐다.한편 복지부는 이번 개정으로 비소세포폐암 환자 치료제 건강보험적용이 가능해져 항암신약에 대한 치료 접근성을 높이고 환자 진료비 부담을 줄일 수 있을 것으로 기대한다고 밝혔다.
사노피-아벤티스 코리아는 국내 최초의 고정비율 통합제제 당뇨병 치료제인 솔리쿠아 펜주가 11월 9일 국내 시판 허가를 받았다고 밝혔다. 국내에서 당뇨병을 치료하는 고정비율 통합제제가 시판 허가를 받은 것은 솔리쿠아가 처음이다.솔리쿠아는 성인 제 2형 당뇨병 환자의 혈당을 조절하는 치료제로, 기존의 메트포르민과 다른 경구 혈당강하제 병용 치료 혹은 기저인슐린 치료로 혈당 조절이 어려운 성인 제 2형 당뇨병 환자의 치료에 사용될 수 있다. 솔리쿠아는 하루 한 번 투여

In [35]:
# 메디컬타임즈 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 0
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.medicaltimes.com/Users4/News/NewsList.html?nStart={}&KeyWord=&smode=&nType=&nSection=&nPlan=&'.format(10*i)
    site = 'http://www.medicaltimes.com/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.find('ul',{'class':'lh22'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages.findAll("a"):
        gisa_links.append(site+page.attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('td',{'colspan':'2','style':'padding: 5px;'}).get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            origin_title = dom.find('td',{'class':'px21 bk fbd lh15 lt'}).get_text()
            re_title = re.sub(title_pattern,'', origin_title)
            title.append(re_title)
            
            # 부제목 크롤링 => 없음
            #stitle = dom.find('td',{'class':'view_sub_t'}).get_text()
            
            # 내용 크롤링
            origin_content = dom.find('td',{'id':'NEWS_CONTENT'}).get_text()
            re_content = re.sub(content_pattern, '', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_medicaltimes = pd.DataFrame(news_dict)

page0
201711
******************************
page1
201711
******************************
page2
201711
******************************
crawling END


In [37]:
# 보건뉴스 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.bokuennews.com/news/article_list_all.html?page={}'.format(i)
    site = 'http://www.bokuennews.com/news/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.findAll('td',{'class':'news1'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages:
        gisa_links.append(site+page.find('a').attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('p',{'class':'arvdate'}).get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('div',{'class':'hbox'}).h2.get_text())
            
            # 부제목 크롤링
            #stitle = dom.find('div',{'class':'hbox'}).h3.get_text()
            
            # 내용 크롤링
            origin_content = dom.find('div',{'id':'news_body_area'}).get_text()
            re_content = re.sub(content_pattern, '', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_bokuennews = pd.DataFrame(news_dict)

page0
201711
******************************
page1
201711
******************************
page2
201711
******************************
crawling END


In [39]:
# 헬스토마토 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.healthtomato.com/subMain.aspx?cate=2201&page={}&type=&val='.format(i)
    site = 'http://www.healthtomato.com/'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.findAll('li',{'class':'news_mlist'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages:
        a = page.find('a').attrs['href'].split('..')
        gisa_links.append(site+a[1])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('div',{'class':'date'}).get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('div',{'class':'ltibox_top'}).b.get_text())
            
            # 부제목 크롤링 => 없음
            #stitle = dom.find('div',{'class':'hbox'}).h3.get_text()
            
            # 내용 크롤링
            origin_content = dom.find('div',{'id':'newsContentArea'}).get_text()
            re_content = re.sub(content_pattern, '', origin_content)
            content.append(re_content)
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_healthtomato = pd.DataFrame(news_dict)

crawling END


In [51]:
# 바이오스펙데이터 크롤링

# 정규 표현식 세팅
date_pattern = '\d\d\d\d[\.-]\d\d[\.-]\d\d'
title_pattern = '\t|(\n)+|\s\s'
content_pattern = '\t|(\n)+|\s\s'

# 크롤링 리스트 모음
date = []
title = []
content = []
link = []

i = 1
flag = True

# 페이지 넘어가면서 크롤링 시작
while flag:
    
    url = 'http://www.biospectator.com/section/section_list.php?MID=10000&page={}'.format(i)
    site = 'http://www.biospectator.com'

    res = requests.get(url)
    dom = BeautifulSoup(res.content, 'html5lib')
    pages = dom.findAll('strong',{'class':'article_tit'})
    
    gisa_links = []


    # 페이지 별 기사링크 크롤링
    for page in pages:
        gisa_links.append(site+page.find('a').attrs['href'])
    
    # 기사링크별 제목, 내용, 날짜 크롤링
    for gisa_link in gisa_links:
        try:
            res = requests.get(gisa_link)
            dom = BeautifulSoup(res.content, 'html5lib')
            
            # 날짜 크롤링
            dates = dom.find('div',{'class':'datetime'}).get_text()
            re_dates = re.search(date_pattern, dates).group()
            date.append(re_dates)
            
            # 제목 크롤링
            title.append(dom.find('div',{'class':'subtit'}).h3.get_text())
            
            # 부제목 크롤링 => 없음
            #stitle = dom.find('div',{'class':'article_view'}).h4.get_text()
            
            # 내용 크롤링
            contents = []
            origin_content = dom.find('div',{'class':'article_view'})
            re_contents = origin_content.findAll('p')
            for re_content in re_contents:
                contents.append(re_content.get_text())
            print(contents[2:-4])
            content.append(contents[2:-4])
            
            # 링크 크롤링
            link.append(gisa_link)
    

        except:
            pass
        
    # 2016년 9월까지 크롤링
    if i>2: 
        #int(re_dates[0:4]+re_dates[5:7]) < int(201609)
        print('crawling END')

        flag = False

    else:
        print('page{}'.format(i))
        print(int(re_dates[0:4]+re_dates[5:7]))
        print('*'*30)
        i += 1
    
# 데이터 리스트 -> 딕트로 변경
news_dict={}
news_dict['date'] = date
news_dict['title'] = title
news_dict['content'] = content
news_dict['link'] = link

# 딕트 -> 데이터프레임으로 변경
df_biospectator = pd.DataFrame(news_dict)

["▲김영덕 더웨이브톡 대표가 10일 서울 양재동 더케이호텔에서 열린 한국식품위생안전성학회에서 실시간 박테리아 검출센서 'Sens.Dr'를 소개하고 있다. 장종원 기자. ", '', "더웨이브톡 연구원이 10^2의 박테리아가 담긴 용액(물)을 실시간 박테리아 검출센서 'Sens.Dr'가 설치된 관으로 주입하자마자 알람과 함께 경고음이 울리기 시작했다. 박테리아 검출여부를 모니터링 하는 모니터에서는 불기둥이 높게 치솟았다. 센서는 물의 유속으로 박테리아 용액이 배출구에 도착하는 시간을 계산해 밸브를 열어 외부로 즉시 배출시켰다. 이 연구원은 10^1, 이어 한 마리의 박테리아가 담긴 용액을 주사기로 주입했고 센서는 그때마다 같은 반응을 했다.", '10일 서울 양재동 더케이호텔에서 열린 한국식품위생안전성학회장. 국내 스타트업 더웨이브톡의 김영덕 대표가 자체 개발한 실시간 박테리아 검출센서 \'Sens.Dr\' 시연에 나섰다. 이 제품은 더웨이브톡의 핵심 기술을 활용해 생수, 음료, 의약품 등을 생산하는 공장의 액체 배송라인에 장착해 실시간으로 박테리아 오염 여부를 확인하고 배출시켜주는 B2B 제품이다. 김 대표는 최근 모 병원의 링거 오염 사례를 거론하며 "히알루론산, 링거액을 생산하는 라인에 적용하면 제품의 오염을 방지해 사고를 막을 수 있다"고도 했다.', '더웨이브톡은 박용근 카이스트 교수가 연구한 ‘시간역전거울’을 활용해 실시간 박테리아 검출 센서를 개발했다. 빛을 쪼였을때 시간역전거울에 반사된 빛의 경로가 박테리아의 움직임때문에 바뀌는 원리를 활용한 것이다. 이를 통해 표본을 채취해 일정시간 배양하거나 공정과정의 온도를 체크해 박테리아 존재여부를 확인하는 기존 방법이 가지는 시간적, 기술적 한계를 획기적으로 개선했다. 수입, 유통, 제조 등의 식품 산업 전반과 다양한 헬스케어 분야에서 박테리아의 유무를 검출하는 것은 중요한 절차 중 하나다. 더웨이브톡은 국내 음료, 주류 기업들과 협력을 모색하고 있다.', "더웨이브톡의 Sens.Dr가 박테리아의 존재 